In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.utils import to_categorical
np.random.seed(178)

In [2]:
csv_file_names = ['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
                  'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
                  'Friday-WorkingHours-Morning.pcap_ISCX.csv',
                  'Monday-WorkingHours.pcap_ISCX.csv',
                  'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
                  'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
                  'Tuesday-WorkingHours.pcap_ISCX.csv',
                  'Wednesday-workingHours.pcap_ISCX.csv']
dfs = []
for file_name in csv_file_names:
    df = pd.read_csv(file_name)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [3]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,53,32215,4,2,112,152,28,28,28.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830739,53,324,2,2,84,362,42,42,42.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830740,58030,82,2,1,31,6,31,0,15.5,21.92031,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830741,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [4]:
df.isnull().sum()

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64

In [5]:
from itertools import combinations

def data_cleaning(df):
    df.columns=df.columns.str.strip()
    print("Dataset Shape: ",df.shape)
    
    num=df._get_numeric_data()
    num[num<0]=0
    
    zero_variance_cols=[]
    for col in df.columns:
        if len(df[col].unique()) == 1:
            zero_variance_cols.append(col)
    df.drop(columns=zero_variance_cols,axis=1,inplace=True)
    print("Zero Variance Columns: ",zero_variance_cols, " are dropped!!")
    print("Shape after removing the zero variance columns: ",df.shape)
    
    df.replace([np.inf,-np.inf],np.nan,inplace=True)
    print(df.isna().any(axis=1).sum(), "rows dropped")
    df.dropna(inplace=True)
    print("Shape after Removing NaN: ",df.shape)
    
    df.drop_duplicates(inplace=True)
    print("Shape after dropping duplicates: ",df.shape)
    
    column_pairs = [(i,j) for i,j in combinations(df,2) if df[i].equals(df[j])]
    ide_cols=[]
    for col_pair in column_pairs:
        ide_cols.append(col_pair[1])
    df.drop(columns=ide_cols,axis=1,inplace=True)
    print("Columns which have identical values: ",column_pairs," dropped!")
    print("Shape after removing identical value columns: ",df.shape)
    return df
df=data_cleaning(df)

Dataset Shape:  (2830743, 79)
Zero Variance Columns:  ['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']  are dropped!!
Shape after removing the zero variance columns:  (2830743, 71)
2867 rows dropped
Shape after Removing NaN:  (2827876, 71)
Shape after dropping duplicates:  (2520798, 71)
Columns which have identical values:  [('Total Fwd Packets', 'Subflow Fwd Packets'), ('Total Backward Packets', 'Subflow Bwd Packets'), ('Fwd PSH Flags', 'SYN Flag Count'), ('Fwd URG Flags', 'CWE Flag Count'), ('Fwd Header Length', 'Fwd Header Length.1')]  dropped!
Shape after removing identical value columns:  (2520798, 66)


In [6]:
df.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',

In [7]:
metadata = ['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Subflow Fwd Bytes', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
       'Idle Std', 'Idle Max', 'Idle Min'] 

In [8]:
# After feature extraction

features = ['Destination Port', 'Total Fwd Packets', 'Total Length of Fwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Mean',
       'Bwd Packet Length Max', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow IAT Max', 'Fwd IAT Std',
       'Fwd Header Length', 'Bwd Header Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'PSH Flag Count', 'Average Packet Size', 'Avg Fwd Segment Size',
       'Avg Bwd Segment Size', 'Subflow Fwd Bytes', 'act_data_pkt_fwd']

In [9]:
df['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

In [10]:
categories = ['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed']

In [11]:
X = df[features]
X[features] = X[features].apply(pd.to_numeric, errors='coerce', axis=1)
X = X.fillna(0)

C:\Users\aryan\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [12]:
X

,Destination Port,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Max,Fwd IAT Std,...,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,PSH Flag Count,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Bytes,act_data_pkt_fwd
0,54865.0,2.0,12.0,6.0,6.0,0.0,0.0,0.0,3.0,0.00000,...,6.0,6.000000,0.000000,0.000000,0.0,9.000000,6.0,0.0,12.0,1.0
1,55054.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,109.0,0.00000,...,6.0,6.000000,0.000000,0.000000,0.0,9.000000,6.0,6.0,6.0,0.0
2,55055.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,52.0,0.00000,...,6.0,6.000000,0.000000,0.000000,0.0,9.000000,6.0,6.0,6.0,0.0
3,46236.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,34.0,0.00000,...,6.0,6.000000,0.000000,0.000000,0.0,9.000000,6.0,6.0,6.0,0.0
4,54863.0,2.0,12.0,6.0,6.0,0.0,0.0,0.0,3.0,0.00000,...,6.0,6.000000,0.000000,0.000000,0.0,9.000000,6.0,0.0,12.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,53.0,4.0,112.0,28.0,28.0,76.0,76.0,0.0,30780.0,17755.84381,...,76.0,41.714286,23.421602,548.571429,0.0,48.666667,28.0,76.0,112.0,3.0
2830739,53.0,2.0,84.0,42.0,42.0,181.0,181.0,0.0,320.0,0.00000,...,181.0,97.600000,76.133435,5796.300000,0.0,122.000000,42.0,181.0,84.0,1.0
2830740,58030.0,2.0,31.0,31.0,15.5,6.0,6.0,0.0,78.0,0.00000,...,31.0,17.000000,16.350331,267.333333,0.0,22.666667,15.5,6.0,31.0,0.0
2830741,53.0,6.0,192.0,32.0,32.0,128.0,128.0,0.0,1000947.0,444210.06860,...,128.0,53.333333,42.332021,1792.000000,0.0,60.000000,32.0,128.0,192.0,5.0


In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2520798 entries, 0 to 2830742
Data columns (total 22 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             float64
 1   Total Fwd Packets            float64
 2   Total Length of Fwd Packets  float64
 3   Fwd Packet Length Max        float64
 4   Fwd Packet Length Mean       float64
 5   Bwd Packet Length Max        float64
 6   Bwd Packet Length Mean       float64
 7   Bwd Packet Length Std        float64
 8   Flow IAT Max                 float64
 9   Fwd IAT Std                  float64
 10  Fwd Header Length            float64
 11  Bwd Header Length            float64
 12  Max Packet Length            float64
 13  Packet Length Mean           float64
 14  Packet Length Std            float64
 15  Packet Length Variance       float64
 16  PSH Flag Count               float64
 17  Average Packet Size          float64
 18  Avg Fwd Segment Size         float64
 19  

In [14]:
y= df['Label']

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(2016638, 22) (2016638,)
(504160, 22) (504160,)


In [16]:
#selecting 1% of random rows for better running time

X_train = X_train.sample(frac=0.1, replace=True, random_state=1)
y_train = y_train.sample(frac=0.1, replace=True, random_state=1)
X_test = X_test.sample(frac=0.1, replace=True, random_state=1)
y_test = y_test.sample(frac=0.1, replace=True, random_state=1)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(201664, 22) (201664,)
(50416, 22) (50416,)


In [17]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_train.values)
X_train = pd.DataFrame(x_scaled,columns=features)
x_scaled_test = min_max_scaler.fit_transform(X_test.values)
X_test = pd.DataFrame(x_scaled_test,columns=features)

In [18]:
encoder = OneHotEncoder()

# Fit and transform the
y_train = encoder.fit_transform(y_train.values.reshape(-1, 1)).toarray()
y_test = encoder.fit_transform(y_test.values.reshape(-1, 1)).toarray()

In [19]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [20]:
y_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [21]:
y_train.shape[1]

15

In [22]:
X_train.values

array([[2.29495689e-01, 4.59145255e-06, 9.31661105e-06, ...,
        0.00000000e+00, 9.31661105e-06, 4.82139155e-06],
       [8.99458305e-01, 0.00000000e+00, 4.65830553e-06, ...,
        1.54745530e-03, 4.65830553e-06, 0.00000000e+00],
       [6.75974670e-03, 1.28560671e-04, 7.85700865e-04, ...,
        2.39364711e-01, 7.85700865e-04, 1.34998963e-04],
       ...,
       [1.22072175e-03, 3.21401679e-05, 2.80274716e-04, ...,
        5.98091472e-01, 2.80274716e-04, 4.82139155e-06],
       [8.08728161e-04, 4.59145255e-06, 5.12413608e-05, ...,
        5.80295736e-02, 5.12413608e-05, 4.82139155e-06],
       [6.75974670e-03, 5.50974306e-05, 5.12413608e-04, ...,
        1.17583156e-01, 5.12413608e-04, 5.78566986e-05]])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(100000, 128, embeddings_regularizer='l2'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(15, activation='sigmoid')
])

model.compile(loss='categorical_crossentropy',
             optimizer=tf.keras.optimizers.Adam(1e-4),
             metrics=['accuracy'])

history = model.fit(X_train.values, y_train, epochs=10, batch_size=32, validation_data=(X_test.values, y_test))# max accuracy i got was 83%

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch 1/10


6302/6302 [==============================] - 1132s 179ms/step - loss: 3.3266 - accuracy: 0.8304 - val_loss: 0.6679 - val_accuracy: 0.8324
Epoch 2/10
6302/6302 [==============================] - 1018s 162ms/step - loss: 0.6718 - accuracy: 0.8311 - val_loss: 0.6420 - val_accuracy: 0.8324
Epoch 3/10
6302/6302 [==============================] - 976s 155ms/step - loss: 0.6550 - accuracy: 0.8311 - val_loss: 0.6355 - val_accuracy: 0.8324
Epoch 4/10
5705/6302 [==========================>...] - ETA: 1:27 - loss: 0.6456 - accuracy: 0.8313